# LABORATORIO 01
## Nombre : Tumiri Huanca Alex
### Adecue la red densamente conectada creada para enfermedades del corazón, para ser utilizada con un dataset de calidad de vinos, que puede descargar de Kaggle.

### Se trabajo con el dataset Calidad de vinos o "winequality-red.csv"
![title](https://vivancoculturadevino.es/blog/wp-content/uploads/2016/03/COPA-VINO-TINTO.jpg)
<a href="https://www.kaggle.com/uciml/red-wine-quality-cortez-et-al-2009/code" target="_blank">Link del Dataset</a>

### Primero Importamos nuestras Librerias necesarias

In [8]:
# Imports
import torch.nn.functional as F  # All functions that don't have any parameters
import pandas as pd
import torch
import torch.optim as optim  # For all Optimization algorithms, SGD, Adam, etc.
import torchvision
import torchvision.transforms as transforms  # Transformations we can perform on our dataset
from pandas import io
from tqdm import tqdm

# from skimage import io
from torch.utils.data import (
    Dataset,
    DataLoader,
)  # Gives easier dataset managment and creates mini batches
import torch.nn as nn  # All neural network modules, nn.Linear, nn.Conv2d, BatchNorm, Loss functions

### Creamos nuestro modelo para evaluar nuestro dataset "winequality-red.csv"

In [2]:
# Create Fully Connected Network
class HearthDiseaseNN(nn.Module):
    def __init__(self, input_size, num_classes):
        super(HearthDiseaseNN, self).__init__()
        self.fc1 = nn.Linear(input_size, 100)
        self.fc2 = nn.Linear(100, num_classes)
        # self.fc2 = nn.Linear(50, 40)
        # self.fc3 = nn.Linear(40, num_classes)

    def forward(self, x):
        x = F.relu(self.fc1(x))
        # x = F.relu(self.fc2(x))
        # x = self.fc3(x)
        x = self.fc2(x)
        return x

class HearthDiseaseDataset(Dataset):
    def __init__(self, csv_file, root_dir, transform=None):
        self.annotations = pd.read_csv(csv_file)
        self.root_dir = root_dir
        self.transform = transform

    def __len__(self):
        return len(self.annotations)

    def __getitem__(self, index):
        x_data = self.annotations.iloc[index, 0:11]
        x_data = torch.tensor(x_data)
        y_label = torch.tensor(int(self.annotations.iloc[index, 11]))

        return (x_data.float(), y_label)

# Set device
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(device)

# Hyperparameters
num_classes = 15
learning_rate = 0.001
batch_size = 400
num_epochs = 700
input_size = 11

# Load Data
dataset = HearthDiseaseDataset(
    csv_file="winequality-red.csv", root_dir="dataset", transform=transforms.ToTensor()
)

train_size = int(0.7 * len(dataset))
test_size = len(dataset) - train_size

train_set, test_set = torch.utils.data.random_split(dataset, [train_size, test_size])
train_loader = DataLoader(dataset=train_set, batch_size=batch_size, shuffle=True)
test_loader = DataLoader(dataset=test_set, batch_size=batch_size, shuffle=True)

# Model
model = HearthDiseaseNN(input_size=input_size, num_classes=num_classes).to(device)

# Loss and optimizer
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=learning_rate)

print(len(train_set))
print(len(test_set))
# Train Network
for epoch in range(num_epochs):
    losses = []

    for batch_idx, (data, targets) in enumerate(train_loader):
        # Get data to cuda if possible
        data = data.to(device=device)
        targets = targets.to(device=device)

        # forward
        scores = model(data)
        loss = criterion(scores, targets)

        losses.append(loss.item())

        # backward
        optimizer.zero_grad()
        loss.backward()

        # gradient descent or adam step
        optimizer.step()

    print(f"Cost at epoch {epoch} is {sum(losses) / len(losses)}")

# Check accuracy on training to see how good our model is
def check_accuracy(loader, model):
    num_correct = 0
    num_samples = 0
    model.eval()

    with torch.no_grad():
        for x, y in loader:
            x = x.to(device=device)
            y = y.to(device=device)

            scores = model(x)
            _, predictions = scores.max(1)
            num_correct += (predictions == y).sum()
            num_samples += predictions.size(0)

        print(
            f"Got {num_correct} / {num_samples} with accuracy {float(num_correct) / float(num_samples) * 100:.2f}"
        )

    model.train()


print("Checking accuracy on Training Set")
check_accuracy(train_loader, model)

print("Checking accuracy on Test Set")
check_accuracy(test_loader, model)



cuda
1119
480
Cost at epoch 0 is 9.67007319132487
Cost at epoch 1 is 7.194334030151367
Cost at epoch 2 is 4.938000361124675
Cost at epoch 3 is 3.251455545425415
Cost at epoch 4 is 2.327177127202352
Cost at epoch 5 is 1.947629451751709
Cost at epoch 6 is 1.6057474613189697
Cost at epoch 7 is 1.5716809034347534
Cost at epoch 8 is 1.6407755215962727
Cost at epoch 9 is 1.482287089029948
Cost at epoch 10 is 1.3301504850387573
Cost at epoch 11 is 1.299149751663208
Cost at epoch 12 is 1.2572667201360066
Cost at epoch 13 is 1.2027517954508464
Cost at epoch 14 is 1.1994701226552327
Cost at epoch 15 is 1.184468428293864
Cost at epoch 16 is 1.1514055331548054
Cost at epoch 17 is 1.1515216827392578
Cost at epoch 18 is 1.1563018560409546
Cost at epoch 19 is 1.1394261916478474
Cost at epoch 20 is 1.1271445353825886
Cost at epoch 21 is 1.1275752782821655
Cost at epoch 22 is 1.120651086171468
Cost at epoch 23 is 1.1145703395207722
Cost at epoch 24 is 1.112658143043518
Cost at epoch 25 is 1.11035720507

Cost at epoch 208 is 0.9192564686139425
Cost at epoch 209 is 0.9118637641270956
Cost at epoch 210 is 0.9181625048319498
Cost at epoch 211 is 0.913665254910787
Cost at epoch 212 is 0.9138383865356445
Cost at epoch 213 is 0.9110968708992004
Cost at epoch 214 is 0.920446773370107
Cost at epoch 215 is 0.9167811075846354
Cost at epoch 216 is 0.9117145339647929
Cost at epoch 217 is 0.9103746811548868
Cost at epoch 218 is 0.9097421964009603
Cost at epoch 219 is 0.9119460781415304
Cost at epoch 220 is 0.9074548284212748
Cost at epoch 221 is 0.9073965549468994
Cost at epoch 222 is 0.9068196018536886
Cost at epoch 223 is 0.9095374743143717
Cost at epoch 224 is 0.9056481122970581
Cost at epoch 225 is 0.9123850067456564
Cost at epoch 226 is 0.9052765369415283
Cost at epoch 227 is 0.9084989428520203
Cost at epoch 228 is 0.9060707092285156
Cost at epoch 229 is 0.9030308127403259
Cost at epoch 230 is 0.9039881030718485
Cost at epoch 231 is 0.9028398990631104
Cost at epoch 232 is 0.9044864177703857
Co

Cost at epoch 413 is 0.8533071676890055
Cost at epoch 414 is 0.8527474602063497
Cost at epoch 415 is 0.8484088977177938
Cost at epoch 416 is 0.8549807866414388
Cost at epoch 417 is 0.8540344834327698
Cost at epoch 418 is 0.8506935834884644
Cost at epoch 419 is 0.8487021327018738
Cost at epoch 420 is 0.851055383682251
Cost at epoch 421 is 0.8468415339787801
Cost at epoch 422 is 0.8499029874801636
Cost at epoch 423 is 0.8469603657722473
Cost at epoch 424 is 0.8482036193211874
Cost at epoch 425 is 0.8483643333117167
Cost at epoch 426 is 0.8510782519976298
Cost at epoch 427 is 0.8475215832392374
Cost at epoch 428 is 0.8530514637629191
Cost at epoch 429 is 0.849335769812266
Cost at epoch 430 is 0.8521290222803751
Cost at epoch 431 is 0.849617063999176
Cost at epoch 432 is 0.8488695621490479
Cost at epoch 433 is 0.8535330096880595
Cost at epoch 434 is 0.8507384856541952
Cost at epoch 435 is 0.8506118853886923
Cost at epoch 436 is 0.8496962587038676
Cost at epoch 437 is 0.8517274459203085
Cos

Cost at epoch 618 is 0.8236630956331888
Cost at epoch 619 is 0.8336813648541769
Cost at epoch 620 is 0.8258795539538065
Cost at epoch 621 is 0.8180985252062479
Cost at epoch 622 is 0.8263124624888102
Cost at epoch 623 is 0.8206165035565695
Cost at epoch 624 is 0.8237271706263224
Cost at epoch 625 is 0.8205190102259318
Cost at epoch 626 is 0.8211658000946045
Cost at epoch 627 is 0.8234320878982544
Cost at epoch 628 is 0.8186495900154114
Cost at epoch 629 is 0.8223796884218851
Cost at epoch 630 is 0.8195895949999491
Cost at epoch 631 is 0.8233983914057413
Cost at epoch 632 is 0.819215714931488
Cost at epoch 633 is 0.8186832666397095
Cost at epoch 634 is 0.8186147014300028
Cost at epoch 635 is 0.8193741242090861
Cost at epoch 636 is 0.821210781733195
Cost at epoch 637 is 0.8184680740038554
Cost at epoch 638 is 0.8185128370920817
Cost at epoch 639 is 0.815855344136556
Cost at epoch 640 is 0.8185677925745646
Cost at epoch 641 is 0.8206358750661215
Cost at epoch 642 is 0.8153817653656006
Cos